In [1]:
import numpy as np
np.seterr(all='raise')
import sys
sys.path.append('../')

from meshmaker.base import Base, Laziness
from meshmaker.model import Model
from meshmaker.mesh import Mesh
from meshmaker.pmesh import ParamMesh, MetaMesh, MetaScene
#from meshmaker.meta import Railing, Stairs
from meshmaker.tform import TForm
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.loop import loop as Loop
from meshmaker.delaunay import triangulation
from meshmaker.planargraph import planargraph
#from meshmaker.geometry import batch, slide, loop_offset, loop_normal, loop_contains, loop_split, isnear, near, loopO
from meshmaker.geometry import batch, slide, isnear, near
from meshmaker.mgl import show, MainShader, EdgeShader, WireShader, LazyMaterials
from meshmaker.plt import *
from collections import defaultdict
from functools import partial, reduce
import json

show = partial(show, programs=[MainShader(), EdgeShader(), WireShader(color=vec3.U(0.1))], background=vec3.U(0.8))

In [2]:
a = Mesh.cube_mesh()
b = Mesh.cube_mesh()
vec3.U(-0.5).trnps(a.vertices)
vec3.U( 0.5).trnps(b.vertices)
show(TForm(models=[Model(meshes={'generic_11': [a], 'generic_10': [b]})]))
#show([a, b])

Loaded texture: generic_11 (../resources/textures/generics/generic_11.png)
Loaded texture: generic_10 (../resources/textures/generics/generic_10.png)


In [3]:
show(a.union(b))
show(b.union(a))
show(a.difference(b))
show(b.difference(a))
show(a.intersect(b))
show(b.intersect(a))

Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


# use CSG to design control meshes which cover 3d volume

# with quat syntax for planar operations?

In [ ]:
class PlanarOperation:
    
    def __init__(self, N, f):
        self.f = f
        self.N = N.cp()
        self.Q = quat.toxy(self.N)
        self.Q_i = quat.toxy(self.N)

    def __call__(self, *ags, **kws):
        for arg in ags:
            self.Q.rot(arg)
        result = self.f(*ags, **kws)
        Qi = self.Q.fp()
        for points in result:
            Qi.rot(points)
        for arg in ags:
            Qi.rot(arg)
        return result

In [ ]:
def planar(operation):
    """Perform an operation on coplanar polygons by transforming to/from the xy-plane
    
    Args:
        operation (function): Function which returns 1+ loops given 1+ loops after
                              applying some operation in the xy-plane.
    
    """
    def wrap(*loops, **kws):
        rot = quat.toxy(loop_normal(loops[0]))
        for loop in loops:
            rot.rot(loop)
        result = operation(*loops, **kws)
        rot = rot.fp()
        for loop in loops:
            rot.rot(loop)
        for loop in result:
            rot.rot(loop)
        return result
    return wrap

In [ ]:


# TODO: wtf is the correct interface of boolean loop operations... (use loop sets to support holes??)

@planar
def loop_intersection(l1, l2):
    """Find the intersection of two loops"""
    l1 = [p.quant() for p in l1]
    l2 = [p.quant() for p in l2]
    segs = list(slide(l1, 2)) + list(slide(l2, 2))
    pg = planargraph(segs=segs)
    _, iloops = pg.polygon()
    parts = []
    for loop in iloops:
        for p in loop:
            if (p.inbxy(l1, True) and p.inbxy(l2, True)):
                continue
            else:
                break
        else:
            parts.append(loop)
    return parts


@planar
def loop_difference(l1, l2):
    
    # to reduce, take 2, return 1
    # list of positives, list of negatives, list of results
    # this probably just cant be reducible... 
    
    #if others:
    #    return reduce(loop_difference, [l1, l2] + list(others))

    print(l1)
    print(l2)

    l1 = [[p.quant() for p in l] for l in l1]
    l2 = [[p.quant() for p in l] for l in l2]
    segs = [list(slide(l, 2)) for l in (l1 + l2)]
    segs = [x for y in segs for x in y]

    #l1 = [p.quant() for p in l1]
    #l2 = [p.quant() for p in l2]
    #segs = list(slide(l1, 2)) + list(slide(l2, 2))
    
    #segs = [(v, u) for u, v in segs]
    pg = planargraph(segs=segs)
    
    #f, ax = plot()
    #plot_pg(ax, pg, lw=5, col='k')
    #for u, v in segs:
    #    plot_edge(ax, u, v, lw=2, col='r')
    
    _, iloops = pg.polygon()
    parts = []
    for loop in iloops:
        for p in loop:
            if not any(p.inbxy(l, True) for l in l2):
                parts.append(loop)
                break
    return parts


@planar
def loop_union(l1, l2):
    l1 = [p.quant() for p in l1]
    l2 = [p.quant() for p in l2]
    segs = list(slide(l1, 2)) + list(slide(l2, 2))
    pg = planargraph(segs=segs)
    union, _ = pg.polygon()
    return [union]


def polygon_difference(py1, py2):
    """py2[1] is within negative space by construction"""
    
    pos = py1[0]
    neg = py1[1] + [py2[0]]
    
    if len(neg) > 1:
        neg = reduce(loop_union, neg)
        
    holes = []
    for hole in neg:
        if loop_contains(pos, hole):
            holes.append(hole)
    for hole in holes:
        neg.remove(hole)

    pos = [pos]
    if len(neg) > 0:
        pos = reduce(loop_difference, pos + neg)
        
    pys = []
    for p in pos:
        pys.append((p, [h for h in holes if loop_contains(p, h)]))

    return pys

In [ ]:
def test():
    l1, l2 = vec3(-1, -1, 0).ring(4, 4), vec3(1, 1, 0).ring(4, 4)

    segs = list(slide(l1, 2)) + list(slide(l2, 2))
    pg = planargraph(segs=segs)
    
    f, ax = plot()
    plot_pg(ax, pg, lw=6, col='g', mk='s')
    #union = loop_union(l1, l2)
    union = [x.ps for x in Loop(l1).union(Loop(l2))]
    for u in union:
        plot_loop(ax, u, lw=3, col='b', mk='s')

    #f, ax = plot()
    #plot_pg(ax, pg, lw=6, col='g', mk='s')
    
    #diff = loop_difference(l1, l2)
    #for d in diff:
    #    plot_loop(ax, d, lw=3, col='b', mk='s')

    f, ax = plot()
    plot_pg(ax, pg, lw=6, col='g', mk='s')
    #intersection = loop_intersection(l1, l2)
    intersection = [x.ps for x in Loop(l1).intersect(Loop(l2))]
    for i in intersection:
        plot_loop(ax, i, lw=3, col='b', mk='s')
    
test()

In [6]:
class Partition(Base):
    
    def __init__(self, *meshes, **kws):
        super().__init__(**kws)
        self.meshcount = 0
        self.meshes = []
        for mesh in meshes:
            self.av(mesh)
    
    def __iter__(self):
        """Yield the index, mesh pairs in the partition"""
        for i, mesh in enumerate(self.meshes):
            if mesh is not None:
                yield (i, mesh)
    
    def av(self, mesh, **kws):
        """Add new volume/vertex"""
        m = len(self.meshes)
        self.meshes.append(mesh)
        self.meshcount += 1
        return m
    
    def rv(self, v):
        """Remove volume/vertex v"""
        mesh = self.meshes[v]
        self.meshes[v] = None
        self.meshcount -= 1
        return mesh
    
    def sv(self, O, N, *vs):
        """Split vertices *vs using plane defined by O and N"""
        nvs = []
        for v in vs:
            v = self.rv(v)
            x, y = v.split(O, N)
            nvs.append(self.av(x))
            nvs.append(self.av(y))
        return nvs
    
    def graph(self):
        """Map the adjacency of volumes within the partition
        identifying the intersections of their bounding faces"""
        adjacent = defaultdict(lambda : defaultdict(lambda : {}))
        normals = {i: mesh.face_normals() for i, mesh in self}
        for i, u in self:
            for k, uF in u:
                uN = normals[i][k]
                uL = [u.vertices[x].cp() for x in uF]
                uL.reverse()
                for j, v in self:
                    if i < j:
                        continue
                    for l, vF in v:
                        vN = normals[j][l]
                        vL = [v.vertices[x].cp() for x in vF]
                        if uN.isnear(vN.fp()):
                            if isnear(uL[0].dot(uN), vL[0].dot(uN)):
                                #overlap = loop_intersection(uL, vL)
                                overlap = [olap.ps for olap in Loop(uL).intersect(Loop(vL))]
                                if overlap:
                                    adjacent[i][j][k] = overlap
                                    adjacent[j][i][l] = overlap
        return adjacent

In [7]:
def Wall(supports):

    def setportals(loop, ploops):
        """Make copies of ploops which are properly coplanar with loop"""
        #lN = loop_normal(loop)
        lN = Loop(loop).N
        aligned = []
        for ploop in ploops:
            ploop = [p.cp() for p in ploop]
            #pN = loop_normal(ploop)
            pN = Loop(ploop).N
            if not lN.isnear(pN):
                ploop.reverse()
            inset = [p.cp() for p in ploop]
            dN = lN * (loop[0].dot(lN) - ploop[0].dot(lN))
            dN.trnps(ploop)
            aligned.append((ploop, inset))
        return aligned

    def meta(control, faces):
        assert len(faces) == 1
        meshes = defaultdict(list)
        for f in faces:
            loop = [control.vertices[v] for v in control.faces[f]]

            #N = loop_normal(loop) * 0.1
            N = Loop(loop).N * 0.1
            ploops = []
            for support in supports:
                portals = support['portals']
                for portal, supp in setportals(loop, portals):
                    ploops.append(portal)
                    mesh = Mesh()
                    mesh.bridge(portal, supp)
                    meshes['generic_14'].append(mesh)

                for sloop in support['support']:
                    dbg = Mesh.cube_mesh(0.02)
                    (vec3.com(sloop) + N).trnps(dbg.vertices)
                meshes['generic_17'].append(dbg)

            mesh = Mesh()

            #mesh.af(loop)
            #for ploop in ploops:
            #    mesh.af(ploop)

            try:
                py = (
                    [p.cp().quant() for p in loop],
                    [[p.cp().quant() for p in ib] for ib in ploops])
                mesh.apy(py, 0.0001, None)
            except:
                print(((loop, ploops), ))

            meshes['generic_19'].append(mesh)

        return TForm(models=[Model(meshes=meshes)])

    return meta


def show_graph(nodes, edges):
    """Represent topology of a graph of mesh nodes"""
    # nodes is a dict of (int, mesh) pairs
    # edges is a list of (i, j) int pairs, indices in nodes
    meshes = defaultdict(list)
    for i, node in nodes.items():
        cube = Mesh.cube_mesh(0.02)
        vec3.com(node.vertices).trnps(cube.vertices)
        meshes['generic_8'].append(cube)
    def wires(tf, nodes=None, edges=None):
        wires = []
        for i, j in edges:
            wires.append((vec3.com(nodes[i].vertices),
                          vec3.com(nodes[j].vertices)))
        return wires        
    meshes[next(meshes.keys().__iter__())][0].wires =\
        partial(wires, nodes=nodes, edges=edges)
    return TForm(models=[Model(meshes=meshes)])


def show_partition(partition):
    adjacent = partition.graph()

    nodes = {}
    for i in adjacent:
        nodes[i] = partition.meshes[i].cp().offset(0.1).fp()
        for k, _ in nodes[i]:
            interfaces = []
            for j in adjacent[i]:
                if k in adjacent[i][j]:
                    support = adjacent[i][j][k]
                    interface = {'support': support,
                                 #'portals': [loop_offset(loop, 0.2) for loop in support]}
                                 'portals': [Loop(loop).offset(0.2).ps for loop in support]}
                    interfaces.append(interface)
            wall = Wall(interfaces)
            wall.__name__ = f'metawall-<{i},{k}>'
            nodes[i].meta[k] = wall
    edges = [(i, j) for i in adjacent for j in adjacent[i]]

    topology = show_graph(nodes, edges)    

    #shell_tex = 'generic_3'
    #shell = BSP.brep_union(*filter(None, self.meshes))
    #shell = shell.offset(-0.1)
    #meshes[shell_tex].append(shell)

    mr = TForm(metas=[MetaMesh(node) for i, node in nodes.items()])
    mr.add(topology)
    ms = MetaScene(mr)
    return ms


r = 4
control = Mesh.cube_mesh(r)
p = Partition()
x = p.av(control)
x, y = p.sv(vec3.Z(1), vec3.Z(), x)
x, z = p.sv(vec3.Z(1), vec3.X(), x)
w, z = p.sv(vec3.Z(1), vec3.Y(), z)
y, a = p.sv(vec3.Z(1), vec3.U().xy().nrm(), y)
p.rv(w)
p.rv(y)
ms = show_partition(p)
show(ms)

Loaded texture: generic_19 (../resources/textures/generics/generic_19.png)
Loaded texture: generic_14 (../resources/textures/generics/generic_14.png)
Loaded texture: generic_17 (../resources/textures/generics/generic_17.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)


# TODO: support layers/groups for showing/hiding meshes in mgl window 

In [ ]:
def explode(*meshes, O=vec3.O(), d=1):
    for mesh in meshes:
        com = vec3.com(mesh.vertices)
        ((com - O) * d).trnps(mesh.vertices)
    return meshes

def compact(*meshes, d=0.1):
    for mesh in meshes:
        com = vec3.com(mesh.vertices)
        com.trnps(vec3.U(1 - d).sclps(com.fp().trnps(mesh.vertices)))
    return meshes    

In [ ]:
a = Mesh.cube_mesh()

u, v = BSP.brep_split(a, Plane(vec3.Z(0.2), vec3(0.2, 0.2, 0.8).nrm()))
u, y = BSP.brep_split(u, Plane(vec3.Z(0.6), vec3(-0.2, 0.2, 0.0).nrm()))
v, w = BSP.brep_split(v, Plane(vec3.X(0.5), vec3.X()))
v, x = BSP.brep_split(v, Plane(vec3.X(0.3), vec3.U().xy()))

show(compact(u, v, w, x, y))

In [ ]:
a = Mesh.cube_mesh(1)

a = BSP.from_brep(a)
b = BSP.from_plane(Plane(vec3.O(), vec3.Z()))
b.build(a.all_loops())

show(compact(*b.breps()))

#show([a, b])